# Lab1-2

In [1]:
import re
import unicodedata
import pandas as pd
import numpy as np
import math
from sklearn.feature_extraction import text
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import classification_report, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from scipy.sparse import hstack
from datetime import time
from sklearn.linear_model import RidgeClassifier, Ridge

Use function which clean URL, EMOJI(unicode only), HASHTAG, EMAIL, CASHTAG, DATE, TIME, EMPHASIS, ELONG

But not MENTION (it's positively affects on result).

Also the function `clean.fit` normalize text: expand contraction, fix typos, etc.

In [2]:
class clean:
    def __init__(self):
        self.regex_dict = {
            'URL': r"""(?xi)\b(?:(?:https?|ftp|file):\/\/|www\.|ftp\.|pic\.|twitter\.|facebook\.)(?:\([-A-Z0-9+&@#\/%=~_|$?!:;,.]*\)|[-A-Z0-9+&@#\/%=~_|$?!:;,.])*(?:\([-A-Z0-9+&@#\/%=~_|$?!:,.]*\)|[A-Z0-9+&@#\/%=~_|$])""",
            'EMOJI': u'([\U0001F1E0-\U0001F1FF])|([\U0001F300-\U0001F5FF])|([\U0001F600-\U0001F64F])|([\U0001F680-\U0001F6FF])|([\U0001F700-\U0001F77F])|([\U0001F800-\U0001F8FF])|([\U0001F900-\U0001F9FF])|([\U0001FA00-\U0001FA6F])|([\U0001FA70-\U0001FAFF])|([\U00002702-\U000027B0])|([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])',
            'HASHTAG': r"\#\b[\w\-\_]+\b",
            'EMAIL': r"(?:^|(?<=[^\w@.)]))(?:[\w+-](?:\.(?!\.))?)*?[\w+-]@(?:\w-?)*?\w+(?:\.(?:[a-z]{2,})){1,3}(?:$|(?=\b))",
            'CASHTAG': r"(?:[$\u20ac\u00a3\u00a2]\d+(?:[\\.,']\d+)?(?:[MmKkBb](?:n|(?:il(?:lion)?))?)?)|(?:\d+(?:[\\.,']\\d+)?[$\u20ac\u00a3\u00a2])",
            'DATE': r"(?:(?:(?:(?:(?<!:)\b\'?\d{1,4},? ?)?\b(?:[Jj]an(?:uary)?|[Ff]eb(?:ruary)?|[Mm]ar(?:ch)?|[Aa]pr(?:il)?|May|[Jj]un(?:e)?|[Jj]ul(?:y)?|[Aa]ug(?:ust)?|[Ss]ept?(?:ember)?|[Oo]ct(?:ober)?|[Nn]ov(?:ember)?|[Dd]ec(?:ember)?)\b(?:(?:,? ?\'?)?\d{1,4}(?:st|nd|rd|n?th)?\b(?:[,\\/]? ?\'?\d{2,4}[a-zA-Z]*)?(?: ?- ?\d{2,4}[a-zA-Z]*)?(?!:\d{1,4})\b))|(?:(?:(?<!:)\b\\'?\d{1,4},? ?)\b(?:[Jj]an(?:uary)?|[Ff]eb(?:ruary)?|[Mm]ar(?:ch)?|[Aa]pr(?:il)?|May|[Jj]un(?:e)?|[Jj]ul(?:y)?|[Aa]ug(?:ust)?|[Ss]ept?(?:ember)?|[Oo]ct(?:ober)?|[Nn]ov(?:ember)?|[Dd]ec(?:ember)?)\b(?:(?:,? ?\'?)?\d{1,4}(?:st|nd|rd|n?th)?\b(?:[,\\/]? ?\'?\d{2,4}[a-zA-Z]*)?(?: ?- ?\d{2,4}[a-zA-Z]*)?(?!:\d{1,4})\b)?))|(?:\b(?<!\d\\.)(?:(?:(?:[0123]?[0-9][\\.\\-\\/])?[0123]?[0-9][\\.\\-\\/][12][0-9]{3})|(?:[0123]?[0-9][\\.\\-\\/][0123]?[0-9][\\.\\-\\/][12]?[0-9]{2,3}))(?!\.\d)\b))",
            'TIME': r'(?:(?:\d+)?\\.?\d+(?:AM|PM|am|pm|a\\.m\\.|p\\.m\\.))|(?:(?:[0-2]?[0-9]|[2][0-3]):(?:[0-5][0-9])(?::(?:[0-5][0-9]))?(?: ?(?:AM|PM|am|pm|a\\.m\\.|p\\.m\\.))?)',
            'EMPHASIS': r"(?:\*\b\w+\b\*)",
            'ELONG': r"\b[A-Za-z]*([a-zA-Z])\1\1[A-Za-z]*\b"}

        self.contraction_mapping = {"’": "'", "RT ": " ", "ain't": "is not", "aren't": "are not", "can't": "can not",
                                    "'cause": "because", "could've": "could have",
                                    "couldn't": "could not", "didn't": "did not", "doesn't": "does not",
                                    "don't": "do not", "hadn't": "had not",
                                    "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'll": "he will",
                                    "he's": "he is",
                                    "how'd": "how did", "how'd'y": "how do you", "how'll": "how will",
                                    "how's": "how is", "I'd": "I would",
                                    "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have", "I'm": "I am",
                                    "I've": "I have",
                                    "i'd": "i would", "i'd've": "i would have", "i'll": "i will",
                                    "i'll've": "i will have", "i'm": "i am",
                                    "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have",
                                    "it'll": "it will",
                                    "it'll've": "it will have", "it's": "it is", "it’s": "it is", "let's": "let us",
                                    "ma'am": "madam", "mayn't": "may not",
                                    "might've": "might have", "mightn't": "might not", "mightn't've": "might not have",
                                    "must've": "must have",
                                    "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not",
                                    "needn't've": "need not have",
                                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
                                    "shan't": "shall not",
                                    "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would",
                                    "she'd've": "she would have",
                                    "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                                    "should've": "should have",
                                    "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                                    "so's": "so as",
                                    "this's": "this is", "that'd": "that would", "that'd've": "that would have",
                                    "that's": "that is",
                                    "there'd": "there would", "there'd've": "there would have", "there's": "there is",
                                    "here's": "here is",
                                    "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
                                    "they'll've": "they will have",
                                    "they're": "they are", "they've": "they have", "to've": "to have",
                                    "wasn't": "was not", "we'd": "we would",
                                    "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                                    "we're": "we are", "we've": "we have",
                                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have",
                                    "what're": "what are",
                                    "what's": "what is", "what've": "what have", "when's": "when is",
                                    "when've": "when have", "where'd": "where did",
                                    "where's": "where is", "where've": "where have", "who'll": "who will",
                                    "who'll've": "who will have",
                                    "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have",
                                    "will've": "will have",
                                    "won't": "will not", "won't've": "will not have", "would've": "would have",
                                    "wouldn't": "would not",
                                    "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                                    "y'all'd've": "you all would have",
                                    "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would",
                                    "you'd've": "you would have",
                                    "you'll": "you will", "you'll've": "you will have", "you're": "you are",
                                    "you've": "you have", "It's": "It is", "You'd": "You would",
                                    ' u ': " you ", 'yrs': 'years', 'FYI': 'For your information', ' im ': ' I am ',
                                    'lol': 'LOL', 'You\'re': 'You are'
            , 'can’t': 'can not', '…': '. ', '...': '. ', '\'\'': '\'', '≠': '', 'ain’t': 'am not', 'I’m': 'I am',
                                    'RT\'s': ''}
        self.emoticons = {
            ':*': '<kiss>',
            ':-*': '<kiss>',
            ':x': '<kiss>',
            ':-)': '<happy>',
            ':-))': '<happy>',
            ':-)))': '<happy>',
            ':-))))': '<happy>',
            ':-)))))': '<happy>',
            ':-))))))': '<happy>',
            ':)': '<happy>',
            ':))': '<happy>',
            ':)))': '<happy>',
            ':))))': '<happy>',
            ':)))))': '<happy>',
            ':))))))': '<happy>',
            ':)))))))': '<happy>',
            ':o)': '<happy>',
            ':]': '<happy>',
            ':3': '<happy>',
            ':c)': '<happy>',
            ':>': '<happy>',
            '=]': '<happy>',
            '8)': '<happy>',
            '=)': '<happy>',
            ':}': '<happy>',
            ':^)': '<happy>',
            '|;-)': '<happy>',
            ":'-)": '<happy>',
            ":')": '<happy>',
            '\o/': '<happy>',
            '*\\0/*': '<happy>',
            ':-D': '<laugh>',
            ':D': '<laugh>',
            '8-D': '<laugh>',
            '8D': '<laugh>',
            'x-D': '<laugh>',
            'xD': '<laugh>',
            'X-D': '<laugh>',
            'XD': '<laugh>',
            '=-D': '<laugh>',
            '=D': '<laugh>',
            '=-3': '<laugh>',
            '=3': '<laugh>',
            'B^D': '<laugh>',
            '>:[': '<sad>',
            ':-(': '<sad>',
            ':-((': '<sad>',
            ':-(((': '<sad>',
            ':-((((': '<sad>',
            ':-(((((': '<sad>',
            ':-((((((': '<sad>',
            ':-(((((((': '<sad>',
            ':(': '<sad>',
            ':((': '<sad>',
            ':(((': '<sad>',
            ':((((': '<sad>',
            ':(((((': '<sad>',
            ':((((((': '<sad>',
            ':(((((((': '<sad>',
            ':((((((((': '<sad>',
            ':-c': '<sad>',
            ':c': '<sad>',
            ':-<': '<sad>',
            ':<': '<sad>',
            ':-[': '<sad>',
            ':[': '<sad>',
            ':{': '<sad>',
            ':-||': '<sad>',
            ':@': '<sad>',
            ":'-(": '<sad>',
            ":'(": '<sad>',
            'D:<': '<sad>',
            'D:': '<sad>',
            'D8': '<sad>',
            'D;': '<sad>',
            'D=': '<sad>',
            'DX': '<sad>',
            'v.v': '<sad>',
            "D-':": '<sad>',
            '(>_<)': '<sad>',
            ':|': '<sad>',
            '>:O': '<surprise>',
            ':-O': '<surprise>',
            ':-o': '<surprise>',
            ':O': '<surprise>',
            '°o°': '<surprise>',
            'o_O': '<surprise>',
            'o_0': '<surprise>',
            'o.O': '<surprise>',
            'o-o': '<surprise>',
            '8-0': '<surprise>',
            '|-O': '<surprise>',
            ';-)': '<wink>',
            ';)': '<wink>',
            '*-)': '<wink>',
            '*)': '<wink>',
            ';-]': '<wink>',
            ';]': '<wink>',
            ';D': '<wink>',
            ';^)': '<wink>',
            ':-,': '<wink>',
            '>:P': '<tong>',
            ':-P': '<tong>',
            ':P': '<tong>',
            'X-P': '<tong>',
            'x-p': '<tong>',
            ':-p': '<tong>',
            ':p': '<tong>',
            '=p': '<tong>',
            ':-Þ': '<tong>',
            ':Þ': '<tong>',
            ':-b': '<tong>',
            ':b': '<tong>',
            ':-&': '<tong>',
            '>:\\': '<annoyed>',
            '>:/': '<annoyed>',
            ':-/': '<annoyed>',
            ':-.': '<annoyed>',
            ':/': '<annoyed>',
            ':\\': '<annoyed>',
            '=/': '<annoyed>',
            '=\\': '<annoyed>',
            ':L': '<annoyed>',
            '=L': '<annoyed>',
            ':S': '<annoyed>',
            '>.<': '<annoyed>',
            ':-|': '<annoyed>',
            '<:-|': '<annoyed>',
            ':-X': '<seallips>',
            ':X': '<seallips>',
            ':-#': '<seallips>',
            ':#': '<seallips>',
            'O:-)': '<angel>',
            '0:-3': '<angel>',
            '0:3': '<angel>',
            '0:-)': '<angel>',
            '0:)': '<angel>',
            '0;^)': '<angel>',
            '>:)': '<devil>',
            '>:D': '<devil>',
            '>:-D': '<devil>',
            '>;)': '<devil>',
            '>:-)': '<devil>',
            '}:-)': '<devil>',
            '}:)': '<devil>',
            '3:-)': '<devil>',
            '3:)': '<devil>',
            'o/\o': '<highfive>',
            '^5': '<highfive>',
            '>_>^': '<highfive>',
            '^<_<': '<highfive>',
            '<3': '<heart>',
            '^3^': '<smile>',
            "(':": '<smile>',
            " > < ": '<smile>',
            "UvU": '<smile>',
            "uwu": '<smile>',
            'UwU': '<smile>'
        }

    def get_compiled(self):
        regexes = {k: re.compile(self.regex_dict[k]) for k, v in
                   self.regex_dict.items()}
        return regexes

    def fit(self, Example):
        regex = self.get_compiled()
        for key, reg in regex.items():
            Example = regex[key].sub(lambda m: " <" + key + "> ",
                                               Example)
        for word in self.emoticons.keys():
            Example = Example.replace(word, self.emoticons[word])
        Example = Example.lower()
        for word in self.contraction_mapping.keys():
            Example = Example.replace(word, self.contraction_mapping[word])
        Example = re.sub(r"[\-\"`@#$%^&*(|)/~\[\]{\}:;+,._='!?]+", " ", Example)
        Example = unicodedata.normalize('NFKD', Example).encode('ascii', errors='ignore').decode('utf8',
                                                                                                 errors='ignore')
        Example = re.sub(r'\b([b-hB-Hj-zJ-Z] )', ' ', Example)
        Example = re.sub(r'( [b-hB-Hj-zJ-Z])\b', ' ', Example)
        Example = ' '.join(Example.split())
        return Example

Preprocessing for predicting organization, use class `clean`

In [3]:
train = pd.read_csv("Train.csv")
organization_vectorizer = text.TfidfVectorizer(analyzer='word', min_df=1)

tweets_texts = train['TweetText'].apply(clean().fit)
feature = organization_vectorizer.fit_transform(tweets_texts)
organization_clf = LinearSVC()
organization_clf.fit(feature, train['Topic'])

LinearSVC()

In [4]:
test = pd.read_csv("Test.csv")
tweets_test_texts = test['TweetText'].apply(clean().fit)
test_feature = organization_vectorizer.transform(tweets_test_texts)
print(classification_report(test['Topic'], organization_clf.predict(test_feature)))

              precision    recall  f1-score   support

       apple       0.93      0.95      0.94        98
      google       0.82      0.70      0.75        79
   microsoft       0.75      0.68      0.71        78
     twitter       0.69      0.83      0.75        87

    accuracy                           0.80       342
   macro avg       0.80      0.79      0.79       342
weighted avg       0.80      0.80      0.80       342



Preprocessing for predicting 1 of 3 sentiments, use predicted organization as feature

In [5]:
encoder = LabelEncoder()
encoder.fit(train['Topic'])
def get_org(tweet_text):
    arr = encoder.transform(organization_clf.predict(organization_vectorizer.transform(tweet_text)))
    return np.asmatrix(arr).transpose()

sentiment_vectorizer = text.TfidfVectorizer()

sentiment_feature = hstack([sentiment_vectorizer.fit_transform(tweets_texts), get_org(tweets_texts)])
sentiment_clf = LinearSVC()
sentiment_clf.fit(sentiment_feature, train['Sentiment'])

LinearSVC()

In [6]:
test_sentiment_feature = hstack([sentiment_vectorizer.transform(tweets_test_texts), get_org(tweets_test_texts)])
print(classification_report(test['Sentiment'], sentiment_clf.predict(test_sentiment_feature)))

              precision    recall  f1-score   support

  irrelevant       0.86      0.90      0.88       105
    negative       0.68      0.55      0.61        49
     neutral       0.77      0.82      0.79       156
    positive       0.58      0.47      0.52        32

    accuracy                           0.77       342
   macro avg       0.72      0.68      0.70       342
weighted avg       0.77      0.77      0.77       342



Predicting sentiment using hour and weekday of post

In [7]:
weekday_train_feature = np.array(train['TweetDate'].apply(lambda s: s.split()[0])).reshape(-1, 1)
hour_train_feature = np.array(train['TweetDate'].apply(lambda s: time.fromisoformat(s.split()[3]).hour)).reshape(-1, 1)

org_encoder_weekday = OneHotEncoder()
org_encoder_hours = OneHotEncoder()
org_encoder_weekday.fit(weekday_train_feature)
org_encoder_hours.fit(hour_train_feature)

text_sentiment_feature = hstack([sentiment_vectorizer.transform(tweets_texts),
                                 org_encoder_weekday.transform(weekday_train_feature),
                                 org_encoder_hours.transform(hour_train_feature)])
clf_sentiment_with_time=LinearSVC()
clf_sentiment_with_time.fit(text_sentiment_feature, train['Sentiment'])

LinearSVC()

In [8]:
weekday_test_feature = np.array(test['TweetDate'].apply(lambda s: s.split()[0])).reshape(-1, 1)
hour_test_feature = np.array(test['TweetDate'].apply(lambda s: time.fromisoformat(s.split()[3]).hour)).reshape(-1, 1)

print(classification_report(test['Sentiment'], clf_sentiment_with_time.predict(hstack([
    sentiment_vectorizer.transform(tweets_test_texts),
    org_encoder_weekday.transform(weekday_test_feature),
    org_encoder_hours.transform(hour_test_feature)]))))


              precision    recall  f1-score   support

  irrelevant       0.88      0.90      0.89       105
    negative       0.76      0.59      0.67        49
     neutral       0.78      0.85      0.81       156
    positive       0.60      0.47      0.53        32

    accuracy                           0.80       342
   macro avg       0.76      0.70      0.72       342
weighted avg       0.79      0.80      0.79       342



### Expand 3 sentiment classes to 5.
Use `sentiment_feature` from first sentiment prediction (by text and organization info) for regression.

In [9]:
clf_sentiment5 = RidgeClassifier()
clf_sentiment5.fit(sentiment_feature, train['Sentiment'])

print(classification_report(test['Sentiment'], clf_sentiment5.predict(test_sentiment_feature)))

d = {'positive': +1, 'neutral': 0, 'negative': -1, 'irrelevant': 0}
sentiment_train_val = train['Sentiment'].apply(lambda s: d[s])
regressor = Ridge()
regressor.fit(sentiment_feature, sentiment_train_val)

sentiment_test_val = test['Sentiment'].apply(lambda s: d[s])
print("mean_absolute_error: ", mean_absolute_error(sentiment_test_val, regressor.predict(test_sentiment_feature)))
print("mean_squared_error: ", mean_squared_error(sentiment_test_val, regressor.predict(test_sentiment_feature)))

              precision    recall  f1-score   support

  irrelevant       0.85      0.89      0.87       105
    negative       0.72      0.53      0.61        49
     neutral       0.76      0.84      0.80       156
    positive       0.50      0.38      0.43        32

    accuracy                           0.77       342
   macro avg       0.71      0.66      0.68       342
weighted avg       0.76      0.77      0.76       342

mean_absolute_error:  0.25502231674475795
mean_squared_error:  0.15936363813354903


Transform from (-1, 1) to {-2, -1, 0, 1, 2}, by splitting to 5 eq segments (with converting values >= 1 to 0.99 and  <= -1 to -0.99)

In [10]:
res5_pred_feature = np.vectorize(lambda x: np.round(max(min(x, 0.99), -0.99) * 2.5))(regressor.predict(test_sentiment_feature))
unique, counts = np.unique(res5_pred_feature, return_counts=True)
print("statistic res:", dict(zip(unique, counts)))

statistic res: {-2.0: 10, -1.0: 58, 0.0: 232, 1.0: 34, 2.0: 8}
